# Find proteins within the event (cis) that are commonly different between samples with and without the event

## Setup

In [1]:
import cnvutils
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
# These variables specify which chromosome and arm we're working on, and whether to do cis or trans effects
params = cnvutils.load_params(os.path.join("..", "data", "params.json"))
CHROMOSOME = params["CHROMOSOME"]
ARM = params["ARM"]
CIS_OR_TRANS = "cis"

read_path = os.path.join("..", "data", f"{CHROMOSOME}{ARM}_ttest_{CIS_OR_TRANS}_effects.tsv")

ttest_results = pd.\
read_csv(read_path, sep="\t").\
rename(columns={"Name": "protein"})

In [3]:
ttest_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,ADAMDEC1,NP_055294.1|NP_001138744.1,6.022237e-01,-0.439719
1,brca,AGPAT5,NP_060831.2,3.624996e-03,-0.849755
2,brca,ANGPT2,NP_001138.1|NP_001112360.1|NP_001112359.1,2.282627e-01,0.490830
3,brca,ARHGEF10,NP_001295082.1|NP_055444.2|NP_001295081.1,2.802092e-02,-0.503329
4,brca,ASAH1,NP_004306.3|NP_808592.2,6.016957e-02,-0.667580
5,brca,ATP6V1B2,NP_001684.2,4.029910e-05,-0.454052
6,brca,BIN3,NP_061158.1,5.095029e-02,-0.552129
7,brca,BLK,NP_001706.2|NP_001317394.1,7.161008e-02,0.537387
8,brca,BMP1,NP_006120.1|NP_001190.1,9.164002e-01,-0.040849
9,brca,BNIP3L,NP_004322.1|NP_001317420.1,3.304251e-02,-0.878878


## Select the proteins with a significant change, and take a detour to make some plots

In [4]:
prots = ttest_results[ttest_results["adj_p"] <= 0.05].reset_index(drop=True)
prots_cts = prots.groupby("cancer_type").count()[["protein"]]

fail_prots = ttest_results[ttest_results["adj_p"] > 0.05].reset_index(drop=True)
fail_cts = fail_prots.groupby("cancer_type").count()[["protein"]]

prots_cts.insert(0, "count_type", "Significant difference")
fail_cts.insert(0, "count_type", "No significant difference")

counts = prots_cts.append(fail_cts).sort_index().reset_index(drop=False)

alt.Chart(counts).mark_bar().encode(
    x=alt.X(
        "count_type",
        axis=alt.Axis(
            title=None,
            labels=False
        ),
        sort=["Significant difference"]
    ),
    y=alt.Y(
        "protein",
        axis=alt.Axis(
            title="Number of proteins"
        )
    ),
    color=alt.Color(
        "count_type",
        title=None,
        sort=["Significant difference"],
        scale=alt.Scale(
            domain=["Significant difference", "No significant difference"],
            range=["#2d3da4", "#d1d1d1"]
        )
    )
).facet(
    column=alt.Column(
        "cancer_type",
        title=None
    )
).properties(
    title=f"Chr {CHROMOSOME}{ARM} {CIS_OR_TRANS} effects"
).configure_title(
    anchor="middle"
)

alt.FacetChart(...)

## Find how many cancers each protein was different in

In [5]:
def make_simple_change(change_val):
    if change_val == 0:
        return 0
    if change_val > 0:
        return 1
    if change_val < 0:
        return -1

prots = prots.assign(
    simplified_change=prots["change"].apply(make_simple_change)
)

In [6]:
prots

,cancer_type,protein,Database_ID,adj_p,change,simplified_change
0,brca,AGPAT5,NP_060831.2,3.624996e-03,-0.849755,-1
1,brca,ARHGEF10,NP_001295082.1|NP_055444.2|NP_001295081.1,2.802092e-02,-0.503329,-1
2,brca,ATP6V1B2,NP_001684.2,4.029910e-05,-0.454052,-1
3,brca,BNIP3L,NP_004322.1|NP_001317420.1,3.304251e-02,-0.878878,-1
4,brca,CHMP7,NP_689485.1|NP_001304828.1,2.897581e-08,-0.779503,-1
5,brca,CNOT7,NP_001309022.1|NP_001309020.1|NP_001309023.1|N...,3.485383e-03,-0.405375,-1
6,brca,DOCK5,NP_079216.4,1.137498e-02,-0.518691,-1
7,brca,ERI1,NP_699163.2,1.115988e-04,-0.853908,-1
8,brca,ERICH1,NP_997215.1|NP_001290029.1,8.967549e-03,-0.691702,-1
9,brca,FAM160B2,NP_073586.5,3.035904e-04,-0.805612,-1


In [7]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist()),
    "mean_simp_change": ("simplified_change", np.mean)
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,mean_simp_change,num_cancers
protein,,,
AGPAT5,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
ATP6V1B2,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
CHMP7,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
ERI1,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
KIF13B,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
MSRA,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
PPP2R2A,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
VPS37A,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6
XPO7,"[brca, colon, hnscc, lscc, luad, ovarian]",-1,6


In [8]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

6     9
5     3
4    13
3     7
2     8
1    17
Name: num_cancers, dtype: int64

## Save results

In [9]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = os.path.join("..", "data", f"common_summary_{CHROMOSOME}{ARM}_{CIS_OR_TRANS}.tsv")
prots_summary.to_csv(output_file, sep="\t")

## Info for most common proteins

- [AGPAT5](https://www.uniprot.org/uniprot/Q9NUQ2)  (colon, hnscc, lscc, luad, ovarian)     5
    - Converts 1-acyl-sn-glycerol-3-phosphate (lysophosphatidic acid or LPA) into 1,2-diacyl-sn-glycerol-3-phosphate (phosphatidic acid or PA) by incorporating an acyl moiety at the sn-2 position of the glycerol backbone (PubMed:21173190). Acts on LPA containing saturated or unsaturated fatty acids C15:0-C20:4 at the sn-1 position using C18:1-CoA as the acyl donor (PubMed:21173190). Also acts on lysophosphatidylethanolamine using oleoyl-CoA, but not arachidonoyl-CoA, and lysophosphatidylinositol using arachidonoyl-CoA, but not oleoyl-CoA (PubMed:21173190). Activity toward lysophosphatidylglycerol not detectable (PubMed:21173190).
- ATP6V1B2 	(colon, hnscc, lscc, luad, ovarian) 	5
- CCDC25 	(colon, hnscc, lscc, luad, ovarian) 	5
- CHMP7 	(colon, hnscc, lscc, luad, ovarian) 	5
- [ERI1](https://www.uniprot.org/uniprot/Q8IV48)    (colon, hnscc, lscc, luad, ovarian)     5
    - RNA exonuclease that binds to the 3'-end of histone mRNAs and degrades them, suggesting that it plays an essential role in histone mRNA decay after replication. A 2' and 3'-hydroxyl groups at the last nucleotide of the histone 3'-end is required for efficient degradation of RNA substrates. Also able to degrade the 3'-overhangs of short interfering RNAs (siRNAs) in vitro, suggesting a possible role as regulator of RNA interference (RNAi). Requires for binding the 5'-ACCCA-3' sequence present in stem-loop structure. Able to bind other mRNAs. Required for 5.8S rRNA 3'-end processing. Also binds to 5.8s ribosomal RNA. Binds with high affinity to the stem-loop structure of replication-dependent histone pre-mRNAs.
- [PPP2CB](https://www.uniprot.org/uniprot/P62714)  (colon, hnscc, lscc, luad, ovarian)     5
    - PP2A can modulate the activity of phosphorylase B kinase casein kinase 2, mitogen-stimulated S6 kinase, and MAP-2 kinase.
- [PPP2R2A](https://www.uniprot.org/uniprot/P63151)     (colon, hnscc, lscc, luad, ovarian)     5
    - The B regulatory subunit might modulate substrate selectivity and catalytic activity, and also might direct the localization of the catalytic enzyme to a particular subcellular compartment.
- [VPS37A](https://www.uniprot.org/uniprot/Q8NEZ2)  (colon, hnscc, lscc, luad, ovarian)     5
    - Component of the ESCRT-I complex, a regulator of vesicular trafficking process. Required for the sorting of endocytic ubiquitinated cargos into multivesicular bodies. May be involved in cell growth and differentiation.
- [XPO7](https://www.uniprot.org/uniprot/Q9UIA9)    (colon, hnscc, lscc, luad, ovarian)  5
    - Mediates the nuclear export of proteins (cargos) with broad substrate specificity. In the nucleus binds cooperatively to its cargo and to the GTPase Ran in its active GTP-bound form. Docking of this trimeric complex to the nuclear pore complex (NPC) is mediated through binding to nucleoporins. Upon transit of a nuclear export complex into the cytoplasm, disassembling of the complex and hydrolysis of Ran-GTP to Ran-GDP (induced by RANBP1 and RANGAP1, respectively) cause release of the cargo from the export receptor. XPO7 then return to the nuclear compartment and mediate another round of transport. The directionality of nuclear export is thought to be conferred by an asymmetric distribution of the GTP- and GDP-bound forms of Ran between the cytoplasm and nucleus.